In [6]:
import pandas as pd
df = pd.read_csv("combined_news_dataset.csv")
df.head()


,id,news_url,title,tweet_ids,label
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,1
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,1
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,1
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,1
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,1


In [7]:
# implementing text cleaning
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

words_stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def text_clean(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','', text)  # Removes the  special characters but keep spaces
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in words_stop]
    return ' '.join(tokens)


df["text_clean"] = df["title"].apply(text_clean)
df["text_clean"].head()

[nltk_data] Downloading package stopwords to C:\Users\Aditya Dwaraki
[nltk_data]     Rao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Aditya Dwaraki
[nltk_data]     Rao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Aditya Dwaraki
[nltk_data]     Rao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0      miley cyrus liam hemsworth secretly get married
1    paris jackson cara delevingne enjoy night matc...
2        celebrity join tax march protest donald trump
3    cindy crawford daughter kaia gerber wear wig d...
4              full list 2018 oscar nomination variety
Name: text_clean, dtype: object

In [8]:
# implementing text cleaning
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download the required NLTK data package
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab') # Download the missing punkt_tab data

words_stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def text_clean(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','', text)  # Removes the  special characters but keep spaces
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in words_stop]
    return ' '.join(tokens)


df["text_clean"] = df["title"].apply(text_clean)
df["text_clean"].head()

[nltk_data] Downloading package stopwords to C:\Users\Aditya Dwaraki
[nltk_data]     Rao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Aditya Dwaraki
[nltk_data]     Rao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Aditya Dwaraki
[nltk_data]     Rao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Aditya Dwaraki
[nltk_data]     Rao\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


0      miley cyrus liam hemsworth secretly get married
1    paris jackson cara delevingne enjoy night matc...
2        celebrity join tax march protest donald trump
3    cindy crawford daughter kaia gerber wear wig d...
4              full list 2018 oscar nomination variety
Name: text_clean, dtype: object

In [9]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_data(data):
    return tokenizer(data,padding = "max_length", truncation = True, max_length = 512, return_tensors = "pt")

df["tokens"] = df["text_clean"].apply(tokenize_data)
df["tokens"].head()


c:\Users\Aditya Dwaraki Rao\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0    [input_ids, token_type_ids, attention_mask]
1    [input_ids, token_type_ids, attention_mask]
2    [input_ids, token_type_ids, attention_mask]
3    [input_ids, token_type_ids, attention_mask]
4    [input_ids, token_type_ids, attention_mask]
Name: tokens, dtype: object

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text_clean"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)

# Tokenizing texts using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
val_encodings = tokenizer(val_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Custom Dataset Class
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Creating DataLoader
train_dataset = FakeNewsDataset(train_encodings, train_labels)
val_dataset = FakeNewsDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [12]:
from transformers import BertForSequenceClassification, AdamW
from transformers import get_scheduler

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer = optimizer, num_warmup_steps=0, num_training_steps=num_training)

#definig the loss function
loss_fn = torch.nn.CrossEntropyLoss()

epochs = 2
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
    train_avg_loss = total_loss / len(train_loader)
    print(f"epoch: {epoch}, train_loss: {train_avg_loss}")
print("Training finished")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


epoch: 0, train_loss: 0.4240556748198538
epoch: 1, train_loss: 0.2820953331470233
Training finished


In [14]:
import sklearn
from sklearn.metrics import classification_report, accuracy_score

model.eval()
y_pred = []
y_true = []

with torch.no_grad():
	for batch in val_loader:
		batch = {k: v.to(device) for k, v in batch.items()}
		outputs = model(**batch)
		logits = outputs.logits
		predictions = torch.argmax(logits, dim=-1)
		y_pred.extend(predictions.cpu().numpy())
		y_true.extend(batch["labels"].cpu().numpy())

print(classification_report(y_true, y_pred))
print(f"Accuracy: {accuracy_score(y_true, y_pred)}")



              precision    recall  f1-score   support

           0       0.89      0.92      0.90      3492
           1       0.72      0.64      0.68      1148

    accuracy                           0.85      4640
   macro avg       0.80      0.78      0.79      4640
weighted avg       0.85      0.85      0.85      4640

Accuracy: 0.8495689655172414


In [17]:
model_pth = "fake_news_detector.pth"
torch.save(model.state_dict(), model_pth)
print("Model saved", model_pth)


Model saved fake_news_detector.pth


In [18]:
# Load model
model.load_state_dict(torch.load("fake_news_detector.pth", map_location=device))
model.to(device)
print("Model loaded successfully!")


C:\Users\Aditya Dwaraki Rao\AppData\Local\Temp\ipykernel_30600\2351491780.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("fake_news_det

Model loaded successfully!


In [19]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def predict_fake_news(text, model, tokenizer, device):
    model.eval()
    
    # Tokenize input text
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    input_ids, attention_mask = inputs["input_ids"].to(device), inputs["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).cpu().numpy()[0]

    return "FAKE NEWS" if prediction == 1 else "REAL NEWS"

# Example News Article
new_article = "The government has announced a new economic policy that will benefit small businesses."
prediction = predict_fake_news(new_article, model, tokenizer, device)
print(f"Prediction: {prediction}")


Prediction: REAL NEWS
